In [1]:
from modules.uploadData import *
from modules.getData import *
from modules.data_setup import create_dataloaders

In [2]:
# zip_data("goose_ducks_dataset")

In [3]:
# get_data_from_github("goose_ducks_dataset")

In [4]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Skalowanie obrazów do 64x64
    transforms.ToTensor(),  # Konwersja obrazu do tensora PyTorch
    transforms.RandomHorizontalFlip(),  # Add horizontal flip
    transforms.RandomRotation(10),  # Add random rotation
])

In [ ]:
train_dataloader, test_dataloader, class_names = create_dataloaders(
    train_dir = "data/goose_ducks_dataset/test",
    test_dir = "data/goose_ducks_dataset/train",
    batch_size=32,
    transform=transform
)

TypeError: create_dataloaders() got an unexpected keyword argument 'transform'

: 

In [ ]:
import torch
from . import model_builder_TinyVGG

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(42)
model = model_builder_TinyVGG.TinyVGG(input_shape=3,
                              hidden_units=10, 
                              output_shape=len(class_names)).to(device)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=0.001)

In [ ]:
from modules.engine import train

train(
    model=model, 
    train_dataloader=train_dataloader, 
    test_dataloader=test_dataloader, 
    optimizer=optimizer,
    loss_fn=loss_fn, 
    epochs=10,
    device=device)

 10%|█         | 1/10 [00:09<01:23,  9.31s/it]

Epoch: 1  | Train Loss: 0.6918  | Train Accuracy: 0.4500  | Test Loss: 0.6792  | Test Accuracy: 0.7708


 20%|██        | 2/10 [00:18<01:13,  9.16s/it]

Epoch: 2  | Train Loss: 0.6881  | Train Accuracy: 0.5500  | Test Loss: 0.6608  | Test Accuracy: 0.7708


 30%|███       | 3/10 [00:27<01:03,  9.07s/it]

...

 90%|█████████ | 9/10 [01:21<00:09,  9.02s/it]

Epoch: 9  | Train Loss: 0.6767  | Train Accuracy: 0.5500  | Test Loss: 0.6046  | Test Accuracy: 0.7708


100%|██████████| 10/10 [01:30<00:00,  9.05s/it]

Epoch: 10  | Train Loss: 0.6736  | Train Accuracy: 0.5500  | Test Loss: 0.6125  | Test Accuracy: 0.7708


{'train_loss': [0.6918157339096069,
  0.6881283521652222,
  0.6845664381980896,
  0.6823102235794067,
  0.6810921430587769,
  0.6808117628097534,
  0.680557906627655,
  0.6795014142990112,
  0.6767256259918213,
  0.673552393913269],
 'train_accuracy': [0.45,
  0.55,
  0.55,
  0.55,
  0.55,
  0.55,
  0.55,
  0.55,
  0.55,
  0.55],
 'test_loss': [0.6791561643282572,
  0.6608231663703918,
  0.6412237286567688,
  0.6223295132319132,
  0.6067568262418112,
  0.5977322061856588,
  0.5952737132708231,
  0.5982956886291504,
  0.6045602560043335,
  0.6125095089276632],
 'test_accuracy': [0.7708333333333334,
  0.7708333333333334,
  0.7708333333333334,
  0.7708333333333334,
  0.7708333333333334,
  0.7708333333333334,
  0.7708333333333334,
  0.7708333333333334,
  0.7708333333333334,
  0.7708333333333334]}

In [ ]:
from modules.utils import save_model

save_model(
    model=model, 
    target_directory="models", 
    model_name="goose_ducks_model_Adam_10Epoch.pth",
    )

Saving model to models\goose_ducks_model_SGDoptim_10Epoch.pth


In [ ]:
import torch
from typing import Dict

def eval_model(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    accuracy_fn
) -> Dict[str, float]:
    model.eval()
    loss, accuracy = 0, 0

    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss += loss_fn(y_pred, y).item()
            accuracy += accuracy_fn(y_pred, y)

    loss /= len(data_loader)
    accuracy /= len(data_loader)
    return {"loss": loss, "accuracy": accuracy}

In [ ]:
from . import model_builder_TinyVGG
from modules.plots import plot_loss_curves
import torch
from modules import utils, engine
import pickle

device = "cuda" if torch.cuda.is_available() else "cpu"

# Define the accuracy function
def accuracy_fn(y_pred, y_true):
    y_pred_class = torch.argmax(y_pred, dim=1)
    correct = (y_pred_class == y_true).sum().item()
    accuracy = correct / len(y_true)
    return accuracy

# Load the saved model
model_path = "models/goose_ducks_model_01.pth"
torch.manual_seed(42)

# Create an instance of the model
class_names = ['ducks', 'goose']  # Ensure class_names is defined
model = model_builder_TinyVGG.TinyVGG(input_shape=3, hidden_units=10, output_shape=len(class_names)).to(device)

# Load the state dictionary into the model
model.load_state_dict(torch.load(model_path))

# Evaluate the loaded model
loaded_model_results = engine.eval_model(
    model=model,
    data_loader=test_dataloader,
    loss_fn=loss_fn, 
    accuracy_fn=accuracy_fn,
    device=device
)

# Print the evaluation results
print(loaded_model_results)

# Assuming you have saved the training history in a file
with open("models/training_history.pkl", "rb") as f:
    history = pickle.load(f)

# Plot the loss curves
plot_loss_curves(history)

c:\Users\gabry\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'test_dataloader' is not defined

In [ ]:
from modules.plots import pred_and_plot_image
import torch
from . import model_builder_TinyVGG

device = "cuda" if torch.cuda.is_available() else "cpu"

model_path = "models/goose_ducks_model_SGDoptim_10Epoch.pth"
torch.manual_seed(42)

model = model_builder_TinyVGG.TinyVGG(input_shape=3,
                              hidden_units=20, 
                              output_shape=len(class_names)).to(device)
model.load_state_dict(torch.load(model_path))

custom_image_transform = transforms.Compose([
    transforms.Resize((64, 64)),
])
custom_image_path = "data/my_ducky.jpg"

pred_and_plot_image(model=model,
                    image_path=custom_image_path,
                    class_names=class_names,
                    transform=custom_image_transform,
                    device=device)

ImportError: attempted relative import with no known parent package

In [ ]:
from modules.plots import pred_and_plot_image
import torch
from . import model_builder_TinyVGG

custom_image_path = "data/my_goose2.jpg"
custom_image_transform = transforms.Compose([
    transforms.Resize((64, 64)),
])

pred_and_plot_image(model=model,
                    image_path=custom_image_path,
                    class_names=class_names,
                    transform=custom_image_transform,
                    device=device)

NameError: name 'transforms' is not defined

In [ ]:
from modules.train import train_model




SyntaxError: invalid syntax (3458882724.py, line 1)